In [3]:
from __future__ import print_function
import argparse
import torch
print(torch.__version__)
import torch.nn as NN
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import Ipynb_importer
from data_loader import DataLoader

1.7.1
importing Jupyter notebook from data_loader.ipynb


In [4]:
use_gpu = torch.cuda.is_available()
use_gpu

True

In [5]:
# Define the model
class Net(NN.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = NN.Conv2d(3, 20, 5, stride=1)
        self.conv2 = NN.Conv2d(20, 50, 5, stride=1)
        self.pool = NN.MaxPool2d(2, 2)
        self.fc1 = NN.Linear(29*29*50, 20)
        self.fc2 = NN.Linear(20, 2)

    def forward(self, x):
        x = self.pool( F.relu(self.conv1(x)) )
        x = self.pool( F.relu(self.conv2(x)) )
        x = x.view(-1, 29*29*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
        
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if (use_gpu):
            model = model.cuda()
            data = data.cuda()
            target = target.cuda()
        optimizer.zero_grad()
        #output = model(torch.autograd.Variable(data))
        output = model( data )
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            if (use_gpu):
                model = model.cuda()
                data = data.cuda()
                target = target.cuda()
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
train_batchsize = 64
test_batchsize = 10
trainEpochs = 10
learningRate = 0.01
sgdmomentum = 0.9

In [7]:
dl = DataLoader("project1/")
trainset = dl.dataset["train"]
testset = dl.dataset["test"]

trainloader = torch.utils.data.DataLoader(trainset, batch_size=train_batchsize, shuffle=True)


testloader = torch.utils.data.DataLoader(testset, batch_size=test_batchsize, shuffle=False)

In [19]:
trainset

Dataset ImageFolder
    Number of datapoints: 1440
    Root location: project1/train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.57794416, 0.57794416, 0.57794416), std=(0.011811608, 0.011811608, 0.011811608))
           )

In [14]:
examples = enumerate(trainloader)
batch_idx, (example_data, example_targets) = next(examples)
example_data

tensor([[[[1.8673, 1.8673, 1.8673,  ..., 1.8673, 1.8673, 1.8673],
          [1.8673, 1.8673, 1.8673,  ..., 1.8673, 1.8673, 1.8673],
          [1.8673, 1.8673, 1.8673,  ..., 1.8673, 1.8673, 1.8673],
          ...,
          [1.8673, 1.8673, 1.8673,  ..., 1.5353, 1.5353, 1.5353],
          [1.8673, 1.8673, 1.8673,  ..., 1.5353, 1.5353, 1.5353],
          [1.8673, 1.8673, 1.8673,  ..., 1.5353, 1.5353, 1.5353]],

         [[1.8673, 1.8673, 1.8673,  ..., 1.8673, 1.8673, 1.8673],
          [1.8673, 1.8673, 1.8673,  ..., 1.8673, 1.8673, 1.8673],
          [1.8673, 1.8673, 1.8673,  ..., 1.8673, 1.8673, 1.8673],
          ...,
          [1.8673, 1.8673, 1.8673,  ..., 1.5353, 1.5353, 1.5353],
          [1.8673, 1.8673, 1.8673,  ..., 1.5353, 1.5353, 1.5353],
          [1.8673, 1.8673, 1.8673,  ..., 1.5353, 1.5353, 1.5353]],

         [[1.8673, 1.8673, 1.8673,  ..., 1.8673, 1.8673, 1.8673],
          [1.8673, 1.8673, 1.8673,  ..., 1.8673, 1.8673, 1.8673],
          [1.8673, 1.8673, 1.8673,  ..., 1

In [25]:
# Train and test
model = Net()
optimizer = optim.SGD(model.parameters(), lr=learningRate, momentum = sgdmomentum)

for epoch in range(trainEpochs):
    train(model, trainloader, optimizer, epoch)
    test(model, testloader)


Train Epoch: 1 [0/1440 (0%)]	Loss: 0.600559

Test set: Average loss: 0.6068, Accuracy: 117/160 (73%)

Train Epoch: 2 [0/1440 (0%)]	Loss: 0.564595

Test set: Average loss: 0.5836, Accuracy: 117/160 (73%)

Train Epoch: 3 [0/1440 (0%)]	Loss: 0.552892

Test set: Average loss: 0.5804, Accuracy: 117/160 (73%)

Train Epoch: 4 [0/1440 (0%)]	Loss: 0.578708

Test set: Average loss: 0.5677, Accuracy: 119/160 (74%)

Train Epoch: 5 [0/1440 (0%)]	Loss: 0.531109

Test set: Average loss: 0.4805, Accuracy: 119/160 (74%)

Train Epoch: 6 [0/1440 (0%)]	Loss: 0.646594

Test set: Average loss: 0.5829, Accuracy: 117/160 (73%)

Train Epoch: 7 [0/1440 (0%)]	Loss: 0.593819

Test set: Average loss: 0.5852, Accuracy: 117/160 (73%)

Train Epoch: 8 [0/1440 (0%)]	Loss: 0.530709

Test set: Average loss: 0.5822, Accuracy: 117/160 (73%)

Train Epoch: 9 [0/1440 (0%)]	Loss: 0.624499

Test set: Average loss: 0.5797, Accuracy: 117/160 (73%)

Train Epoch: 10 [0/1440 (0%)]	Loss: 0.670428

Test set: Average loss: 0.5750, Accu

In [ ]:
torch.save(model.state_dict(),"mnist_cnn.pt")